# Phase 5: Insight Generation and Strategic Recommendations

This notebook generates actionable business insights:
1. Total ROI Decomposition (Short-term + Long-term)
2. Budget Optimization (Maximize ROI)
3. Channel Comparison and Prioritization
4. Strategic Recommendations

**Prerequisites:** Run notebooks 02 and 03 first to calculate ROIs

## 1. Holistic ROI Decomposition

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

# Example ROI values (replace with actual values from notebooks 02 & 03)
marketing_channels = ['Content Marketing', 'Events', 'Google Ads', 'LinkedIn']

# These would come from your fitted models
short_term_roi = {
    'Content Marketing': 0.00,
    'Events': 0.00,
    'Google Ads': 0.00,
    'LinkedIn': 0.00
}

long_term_roi = {
    'Content Marketing': 566.23,
    'Events': 386.09,
    'Google Ads': 402.08,
    'LinkedIn': 1195.11
}

# Calculate total ROI
total_roi = {}
for channel in marketing_channels:
    total_roi[channel] = short_term_roi[channel] + long_term_roi[channel]

print("="*70)
print("TOTAL ROI DECOMPOSITION")
print("="*70)
print(f"\n{'Channel':<20s} {'Short-Term':>12s} {'Long-Term':>12s} {'Total ROI':>12s}")
print("-"*70)

for channel in marketing_channels:
    st = short_term_roi[channel]
    lt = long_term_roi[channel]
    total = total_roi[channel]
    print(f"{channel:<20s} ${st:>11,.2f} ${lt:>11,.2f} ${total:>11,.2f}")

print("\nKey Insight:")
print(f"  Nearly 100% of ROI comes from long-term brand-building effects!")
print(f"  This indicates sustained brand equity drives sales more than immediate activation.")

## 2. Budget Optimization

In [ ]:
# Budget Optimization

print("\n" + "="*70)
print("BUDGET OPTIMIZATION")
print("="*70)

# Current average spend per channel (example values - replace with actual)
current_spend = {
    'Content Marketing': 5000,
    'Events': 2000,
    'Google Ads': 8000,
    'LinkedIn': 10000
}

total_budget = sum(current_spend.values())  # Keep same total budget

print(f"\nCurrent total budget: ${total_budget:,.0f}")
print(f"Objective: Maximize total return while reallocating budget")

# Optimization function
def optimize_budget(total_roi_dict, current_spend_dict, total_budget):
    """
    Find optimal allocation to maximize return with diminishing returns.
    
    Model: return = roi * spend^0.7 (diminishing returns)
    """
    channels = list(total_roi_dict.keys())
    roi_array = np.array([total_roi_dict[ch] for ch in channels])
    
    # Objective: maximize return (minimize negative return)
    def negative_return(spend):
        return -np.sum(roi_array * np.power(spend, 0.7))
    
    # Constraints: spend exactly total_budget
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - total_budget}
    ]
    
    # Bounds: each channel gets 5-60% of budget
    bounds = [(total_budget * 0.05, total_budget * 0.60) for _ in channels]
    
    # Initial guess: proportional to ROI
    roi_weights = roi_array / roi_array.sum()
    x0 = roi_weights * total_budget
    
    # Optimize
    result = minimize(
        negative_return,
        x0=x0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000}
    )
    
    if not result.success:
        print(f"⚠ Optimization warning: {result.message}")
    
    optimal_spend = dict(zip(channels, result.x))
    expected_return = -result.fun
    
    return optimal_spend, expected_return

# Run optimization
optimal_spend, expected_return = optimize_budget(total_roi, current_spend, total_budget)

print(f"\nExpected return with optimal allocation: ${expected_return:,.0f}")
print(f"Expected ROI: ${expected_return/total_budget:.2f} per $1")

print(f"\n{'Channel':<20s} {'Current':>12s} {'Optimal':>12s} {'Change':>12s}")
print("-"*70)

for channel in marketing_channels:
    current = current_spend[channel]
    optimal = optimal_spend[channel]
    change_pct = (optimal - current) / current * 100 if current > 0 else 0
    print(f"{channel:<20s} ${current:>11,.0f} ${optimal:>11,.0f} {change_pct:>+10.1f}%")

print("\nRecommendations:")
for channel in marketing_channels:
    change_pct = (optimal_spend[channel] - current_spend[channel]) / current_spend[channel] * 100
    if change_pct > 10:
        print(f"  ✓ INCREASE {channel} by {change_pct:.0f}% (high ROI)")
    elif change_pct < -10:
        print(f"  ✓ DECREASE {channel} by {abs(change_pct):.0f}% (lower ROI)")
    else:
        print(f"  → MAINTAIN {channel} (near optimal)")

## 3. Scenario Planning & Forecasting

In [ ]:
# Visualize Budget Optimization

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Current vs Optimal Allocation
x = np.arange(len(marketing_channels))
width = 0.35

current_vals = [current_spend[ch] for ch in marketing_channels]
optimal_vals = [optimal_spend[ch] for ch in marketing_channels]

ax1.bar(x - width/2, current_vals, width, label='Current', 
        color='#A23B72', alpha=0.7)
ax1.bar(x + width/2, optimal_vals, width, label='Optimal', 
        color='#18A558', alpha=0.7)

ax1.set_xlabel('Marketing Channel', fontsize=12, fontweight='bold')
ax1.set_ylabel('Budget Allocation ($)', fontsize=12, fontweight='bold')
ax1.set_title('Budget Allocation: Current vs Optimal', 
              fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(marketing_channels, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: ROI vs Budget Change
change_pct = [(optimal_spend[ch] - current_spend[ch]) / current_spend[ch] * 100 
              for ch in marketing_channels]
roi_values = [total_roi[ch] for ch in marketing_channels]
colors_change = ['#18A558' if c > 0 else '#E63946' for c in change_pct]

ax2.barh(marketing_channels, change_pct, color=colors_change, alpha=0.7)
ax2.axvline(0, color='black', linewidth=1, linestyle='--')
ax2.set_xlabel('Budget Change (%)', fontsize=12, fontweight='bold')
ax2.set_title('Recommended Budget Changes', 
              fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

# Add ROI labels
for i, (ch, roi) in enumerate(zip(marketing_channels, roi_values)):
    ax2.text(change_pct[i], i, f'  ROI: ${roi:.0f}', 
            va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 4. Reporting and Visualization

In [ ]:
# Strategic Recommendations Summary

print("\n" + "="*70)
print("STRATEGIC RECOMMENDATIONS")
print("="*70)

# Rank channels by total ROI
ranked_channels = sorted(total_roi.items(), key=lambda x: x[1], reverse=True)

print("\n1. CHANNEL PRIORITIZATION (by Total ROI)")
print("-" * 70)
for rank, (channel, roi) in enumerate(ranked_channels, 1):
    print(f"   {rank}. {channel:<20s} ${roi:>10,.2f} per $1")

print("\n2. KEY INSIGHTS")
print("-" * 70)
best_channel = ranked_channels[0][0]
best_roi = ranked_channels[0][1]
worst_channel = ranked_channels[-1][0]
worst_roi = ranked_channels[-1][1]

print(f"   • Best performer: {best_channel} (${best_roi:.0f} ROI)")
print(f"   • Lowest performer: {worst_channel} (${worst_roi:.0f} ROI)")
print(f"   • ROI gap: {best_roi / worst_roi:.1f}x difference")

# Long-term vs short-term
lt_total = sum(long_term_roi.values())
st_total = sum(short_term_roi.values())
lt_pct = lt_total / (lt_total + st_total) * 100 if (lt_total + st_total) > 0 else 0

print(f"\n   • Long-term effects: {lt_pct:.1f}% of total ROI")
print(f"   • This suggests brand-building is PRIMARY driver of sales")
print(f"   • Focus on sustained campaigns vs short-term activation")

print("\n3. ACTION ITEMS")
print("-" * 70)

# Budget recommendations
for channel in marketing_channels:
    change = optimal_spend[channel] - current_spend[channel]
    change_pct = change / current_spend[channel] * 100
    
    if change_pct > 10:
        print(f"   ✓ {channel}:")
        print(f"     - INCREASE budget by ${change:,.0f} (+{change_pct:.0f}%)")
        print(f"     - High ROI: ${total_roi[channel]:,.0f} per $1")
    elif change_pct < -10:
        print(f"   ⚠ {channel}:")
        print(f"     - DECREASE budget by ${abs(change):,.0f} ({change_pct:.0f}%)")
        print(f"     - Reallocate to higher ROI channels")

print("\n4. IMPLEMENTATION TIMELINE")
print("-" * 70)
print("   Phase 1 (Month 1-2): Test 20% of recommended changes")
print("   Phase 2 (Month 3-4): Scale to 50% if results positive")
print("   Phase 3 (Month 5-6): Full implementation of optimal allocation")
print("   Phase 4 (Ongoing): Monitor and re-optimize quarterly")

print("\n5. SUCCESS METRICS")
print("-" * 70)
print("   • Overall ROI improvement")
print("   • Brand metric lift (Awareness, Consideration)")
print("   • Sales growth rate")
print("   • Customer acquisition cost reduction")

print("\n" + "="*70)
print("✅ INSIGHT GENERATION COMPLETE")
print("="*70)
print("\nNext steps:")
print("  1. Present findings to stakeholders")
print("  2. Get approval for budget reallocation")
print("  3. Implement phased rollout")
print("  4. Monitor results and re-run models quarterly")